# Como usar

Esse aplicativo web é divido em 3 partes:
1. **Gerar o seu sistema:** Isso pode ser feito após entrar as informações do domínio da função de onda $\Psi(x)$, o número de pontos utilizados para representá-la, a forma da **energia potencial $V(x)$** e clicar em `Gerar`.
    * Existem energias potenciais $V(x)$ pré definidas dentro do menu `Potenciais pré definidos`. Se escolher uma opção pré definida, o domínio e o número de pontos serão selecionados automaticamente e, em seguida, o botão 'Gerar' será clicado.
    * A expressão para $V(x)$ está em eletron-volts, e $x$ estará em unidades Raio de Bohr (veja sobre as unidades de Rydberg abaixo).
    * Caso você deseje utilizar $x$ em , divida-o sempre por `ang`. Por exemplo, expressões de $V(x)$ como:
        * `(x - 10)**2` resultará em um potencial harmônico em eletron-volts, centrado em $x=10$ Angstroms.
        * `(x*ang - 10)**2` resultará em um potencial harmônico em eletron-volts, centrado em $x=10$ Raio de Bohr.
    * A energia potencial deve ser da ordem de alguns eV's apenas. 

2. **Diagonalizar a matriz do sistema:** Após o sistema ser gerado, o menu `Informações do Sistema Gerado/Diagonalizar` exibirá uma tabela com algumas informações sobre o sistema gerado. Se estiver tudo como você quer, clique em `Diagonalizar` para resolver a matriz correspondente ao sistema, gerando as auto funções e os auto valores.

3. **Plotar o resultado:** Após exibida a mensagem `Diagonalizado!` no próprio botão `Diagonalizar`, você será direcionado para a aba `Funções de onda`, onde as densidades de probabilidades dos 11 primeiros estados serão plotados automaticamente.

***

In [1]:
%matplotlib widget
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt
from numba import jit
import ipywidgets as wid
from ipywidgets import interactive,fixed, interactive_output

plt.rc('font',size=10)
RY=13.60566 #x*RY: Rydberg -> ev
ang=1.88972 #x/ang: raio de bohr -> angstroms
pot_exemplos=['(2e-4)*((x)**2)','0',
             '(1e-5)*(x**2)+(1e-7)*(x**4)',
             '(10)*( (49.2 / (x+84) )**12 - (49.2 / (x+84) )**6)',
             '3.5*(np.exp(-2*(0.1*x + 2.75)) - 2*np.exp(-(0.1*x + 2.75)))',
             '-4*(np.cosh(0.1*x)**(-2))',
             '0.05*abs(x)-1.5',
             '(1e-5)*(x**2)-(2e-7)*(x**4)',
             '0.05*x',
             '-0.05*x',
             '4*np.sin(-2*np.abs(x)*np.pi/(35))']

################################STYLES/LAYOUT###############################
lslider=wid.Layout(width='95%', display='flex')
lbut=wid.Layout(width='95%', height='50 px')
l100=wid.Layout(width='99%')
lbmargin=wid.Layout(border='solid 2px gray',
        margin='10px 0px 10px 0px',
        padding='5px 5px 5px 5px')
sty = {'description_width': 'initial'}

################WIDGETS###########
xx=wid.FloatRangeSlider(
    value=[-35.0, 35.0],
    min=-150.0,
    max=150.0,
    step=1,
    continuous_update=False,
    orientation='horizontal',
    readout_format='1d',
    layout=lslider,
    description='Domínio de $\Psi(x)$: ',
    style=sty)
N=wid.IntSlider(value=551,
    min=31,
    max=1051,
    step=5,
    continuous_update=False, 
    layout=lslider,
    description=('Número de pontos: '),
    style=sty)
potV=wid.Text(description='Expressão de $V(x)$: ',
    value="0",
    placeholder='Escreva a expressão aqui.',
    style=sty,
    continuous_update=False,
    layout=lslider)
gerar=wid.Button(description='Gerar',button_style='info')
outtsys=wid.Output(layout=lbmargin)
outt=wid.Output()
outtxt=wid.Output(layout=lbmargin)
################FUNCS##############
def diagg(N):
    global x,psi,prob,eig,V
    
    @jit
    def diagH(H,V):
        H=H+V
        eig, psi=np.linalg.eigh(H) #Resolve o sistema e atribui-se: eig=autovalores, psi=autovetores
        prob=np.abs(psi)**2 #Atribui a densidade de probabilidade |Psi|² a variável prob
        return H,V,psi,prob,eig
    
    a=- 1/(dx**2) #Define 'a' por conveniência [1/(raio de Bohr)²]
    H=sparse.diags([a,-2*a,a],[-1,0,1],shape=(N,N),dtype=np.complex128).toarray() #Cria a matriz energia cinética (Rydberg)
    H,V,psi,prob,eig=diagH(H,V)
    V=np.diag(V) #transforma V(x) em um vetor linha

def syst(xx,N,potV):   
    global x,dx,V
    def vetores(xx,N): #Define vetor posicao x (angstroms)
        xmin, xmax=xx #define variaveis para o minimo e maximo do dominio (angstroms)
        x,dx=np.linspace(xmin, xmax, N, retstep=True, dtype=np.float64) #Define o vetor posição (angstroms)
        return x,dx,xmin,xmax
    
    def pot(N,pot): #Define vetor V(X) (eV)
        V=np.ones(shape=(N)) #define o vetor linha V
        V=V*eval(pot) #Define V(x) de acordo com a expressão do usuário (eV)
        return V
    
    x, dx, xmin, xmax = vetores(xx,N) #define vetor posicao (angstrom)
    L = xmax-xmin #define o tamanho do dominio (angstroms)
    
    V = pot(N,potV) #define o vetor potencial V(x) (eV)
    Vlim = [np.min(V), np.max(V)] #define minimo e maximo de V (eV)
    vxmin = x[np.argwhere(V==Vlim[0])] #define posicoes dos maximos de V (angstroms)
    vxmax = x[np.argwhere(V==Vlim[1])] #define posicoes dos minimos de V (angstroms)
    infofill(L,dx,Vlim,vxmin,vxmax,N) #preenche a caixa de informacoes do sistema
    
    outtsys.clear_output() #deleta graficos pré-existentes do sistema
    
    with outtsys: #plota o sistema
        figsys, ax1=plt.subplots(constrained_layout=True) #define a figura e o eixo do plot do sistema (figsys, ax1)
        figsys.canvas.toolbar_position='bottom' #coloca a toolbar interativa embaixo do plot
        figsys.canvas.header_visible = False #tira o header do plot
        figsys.canvas.footer_visible = False #tira o footer do plot
        figsys=plt.xlabel('$x (\AA)$') #titulo eixo X
        figsys=plt.ylabel('Potencial (eV)') #titulo eixo Y
        figsys=plt.plot(x,V,'r-',label='$V(x)$') #plota o potencial V(x)
        figsys=plt.axhline(Vlim[0], xmin, xmax,color='r',ls='--',label='limites de $V(x)$') #plota linha y=Vmin
        figsys=plt.axhline(Vlim[1], xmin, xmax,color='r',ls='--') #plota linha y=Vmax
        figsys=plt.axvline(x=xmin,color='k',ls='--',label='limites de $x$') #plota linha vertical x=xmin
        figsys=plt.axvline(x=xmax,color='k',ls='--',) #plota linha vertical x=xmax
        figsys=plt.setp(ax1, xticks=[xmin, xmax], yticks=Vlim) #coloca os 'ticks' do plot
        leg1=plt.legend(loc='upper right') #plota a legenda na direita superior do plot
    V=np.diag(V) #transforma o vetor linha de N elementos V(x) em uma matriz V(x) NxN
    x, dx, V = x*ang, dx*ang, V/RY

def infofill(L,dx,Vlim,vxmin,vxmax,N):
    outtxt.clear_output()
    vxmin=np.array2string(vxmin, precision=2, suppress_small=True)
    vxmax=np.array2string(vxmax, precision=2, suppress_small=True)
    with outtxt:
        print(f"Distância dx: {dx:.4f}")
        print(f"Número de pontos: {N: 1d}")
        print(f"Largura do domínio: {L: .2f}")
        print(f"Expressão de V(x): " + potV.value)
        print(f"Mínimo e máximo de V(x): {Vlim[0]:.5f}, {Vlim[1]:.5f}")
        print(f"Locais dos mínimo(s) de V(x): " + vxmin)
        print(f"Locais dos máximo(s) de V(x): " + vxmax)
    
    
    
def plotall(NES,limx,wav,dpi,limy,autolim):
    outt.clear_output()
    with outt:
        fig, ax=plt.subplots(constrained_layout=True)
        fig.canvas.toolbar_position='bottom'
        fig.canvas.header_visible = False
        fig.canvas.footer_visible = False
        nmin,nmax = NES
        En=eig*RY
        fig=plt.xlim([x.min()/ang-limx, x.max()/ang +limx])
        fig=plt.xlabel('$x (\AA)$')
        fig=plt.ylabel('Auto energias (eV)')
        fig=plt.title(f"$\Delta x$ = {dx/ang:.4f} | Ordem da matriz = {N.value} x {N.value}")
        probb=norm(prob,nmin,nmax,En,1)
        
        if wav==1:
            for i in range(nmax,nmin-1,-1):
                fig=plt.plot(x/ang,probb[:,i]+En[i], 
                             label=f'$E_{{{i}}}={En[i]:.5f}$ eV')
        elif wav==2:
            for i in range(nmax,nmin-1,-1):
                fig=plt.plot(x/ang,np.real(psi[:,i])+En[i], 
                             label=f'$E_{{{i}}}={En[i]:.5f}$ eV')
        elif wav==3:
            for i in range(nmax,nmin-1,-1):
                fig=plt.plot(x/ang,np.imag(psi[:,i])+En[i], 
                             label=f'$E_{{{i}}}={En[i]:.5f}$ eV')
        if autolim==False:
            fig=plt.ylim(limy)
        
        fig=plt.axvline(x=x.min()/ang,color='k',ls='--')
        fig=plt.axvline(x=x.max()/ang,color='k',ls='--')
        fig=plt.plot(x/ang,V*RY,'k',label='V($x$)')
        leg=plt.legend(loc='upper right')

@jit
def norm(prob,nmin,nmax,En,wav):
    mmax=np.zeros(nmax+2)
    if wav==1:
        for i in range(0,nmax+2):
            mmax[i]=(En[i+1]-En[i])*0.9
        for i in range(0,nmax+2):
            prob[:,i]=prob[:,i]*np.abs(mmax[i]/prob[:,i].max())
    #else:
    #    for i in range(1,nmax+2):
    #        mmax[i]=(prob[:,i+1].min()-prob[:,i-1].max() \
    #                /(prob[:,i].max()-prob[:,i].min()))
    #    for i in range(1,nmax+2):
    #        prob[:,i-1]=prob[:,i-1]*np.abs(mmax[i]*0.9)
    return prob

def potlistclick(bo):
    b=bo.description
    if b=='Oscilador Harmônico':
        potV.value=pot_exemplos[0]
    elif b=='Poço Infinito':
        potV.value=pot_exemplos[1]
    elif b=='Duffing Holmes (+)':
        potV.value=pot_exemplos[2]
    elif b=='Lennard Jones':
        potV.value=pot_exemplos[3]
    elif b=='Morse':
        potV.value=pot_exemplos[4]
    elif b=='Pöschl–Teller':
        potV.value=pot_exemplos[5]
    elif b=='Poço Triangular':
        potV.value=pot_exemplos[6]
    elif b=='Duffing Holmes (-)':
        potV.value=pot_exemplos[7]
    elif b=='Campo Uniforme (-)':
        potV.value=pot_exemplos[8]
    elif b=='Campo Uniforme (+)':
        potV.value=pot_exemplos[9]
    gerarclick(None)

def gerarclick(a):
    if a==None:
        XX=[-35,35]
    else:
        XX=xx.value
    DIAG.disabled=False
    DIAG.description='Diagonalizar'
    DIAG.button_style='info'
    syst(XX,N.value,potV.value)
    PLOT.children[6].disabled=True
    PLOT.children[6].description=('Diagonalize...')
    ACCORDS.selected_index=1
    PLOT.children[6].button_style=''

def diagonalizarclick(a):
    DIAG.description='Diagonalizando...'
    PLOT.children[6].disabled=False
    PLOT.children[6].description=('Plotar')
    diagg(N.value)
    PLOT.children[6].click()
    DIAG.disabled=True
    DIAG.description='Diagonalizado!'
    DIAG.button_style=''
    tab.selected_index=1
    PLOT.children[6].button_style='info'

############################INTERACTIVES################################ 
PLOT=interactive(plotall, {'manual': True},
    NES=wid.IntRangeSlider(value=[0, 10],
        min=0,max=15,
        step=1,
        description='Plotar estados:',
        continuous_update=False,
        orientation='horizontal',
        layout=lslider,
        style=sty),
    limx=wid.IntSlider(value=20,
        min=0,max=100,
        step=1,
        description='Margens horizontais  (angstroms):   ',
        continuous_update=False,
        layout=lslider,
        style=sty),
    wav=wid.Dropdown(
        options=[('densidades de probabilidades',1),
                 ('funções de onda (parte real, +)',2),
                 ('funções de onda (parte imaginária)',3)],
        value=1,
        description='Plotar as',
        style=sty),
    dpi=wid.IntSlider(value=80,
        min=10,max=300,
        step=1,
        description='Pixels por polegada:',
        layout=lslider,
        style=sty),
    limy=wid.FloatRangeSlider(value=[-2.,2.],
        min=-10.,max=10.,
        step=0.005,
        description='Limites do eixo y (eV):',
        orientation='horizontal',
        readout_format='.1f',
        layout=lslider,
        style=sty),
    autolim=wid.Checkbox(value=True,
        description='Auto limitar eixo y',
        style=sty),layout=wid.Layout(border="dashed 2px gray"))

################################BOTOES################################
DIAG=wid.Button(description='Diagonalizar',disabled=True,layout=lbut,
        button_style='info')
OHS=wid.Button(description='Oscilador Harmônico')
POCO=wid.Button(description='Poço Infinito')
DUFHOLMES=wid.Button(description='Duffing Holmes (+)', style=sty)
MDUFHOLMES=wid.Button(description='Duffing Holmes (-)', style=sty)
LENJON=wid.Button(description='Lennard Jones', style=sty)
MORSE=wid.Button(description='Morse', style=sty)
POSTEL=wid.Button(description='Pöschl–Teller', style=sty)
TRIANG=wid.Button(description='Poço Triangular', style=sty)
CAMPMENOS=wid.Button(description='Campo Uniforme (-)', style=sty)
CAMPMAIS=wid.Button(description='Campo Uniforme (+)', style=sty)

################################ON_CLICKS#############################
DIAG.on_click(diagonalizarclick)
OHS.on_click(potlistclick)
POCO.on_click(potlistclick)
DUFHOLMES.on_click(potlistclick)
MDUFHOLMES.on_click(potlistclick)
LENJON.on_click(potlistclick)
MORSE.on_click(potlistclick)
POSTEL.on_click(potlistclick)
TRIANG.on_click(potlistclick)
CAMPMAIS.on_click(potlistclick)
CAMPMENOS.on_click(potlistclick)
gerar.on_click(gerarclick)

###############################labels###################
leg_info=wid.Label(f'Nesses dados e nesse gráfico, todos os valores \
de comprimento estão em angstroms e \
todos os valores de energia estão em eletron-volts.',
layout=wid.Layout(display='flex', justify_content='center'))

################################BOXES###############################
POTSBUT=wid.GridBox([POCO,TRIANG,CAMPMAIS,CAMPMENOS,OHS,DUFHOLMES,
MDUFHOLMES,LENJON,MORSE,POSTEL],
        layout=wid.Layout(grid_template_columns="repeat(auto-fill, 180px)",
        margin='10px 0px 10px 0px',
        padding='5px 5px 5px 5px'))
POTSOPT=wid.VBox([POTSBUT],layout=wid.Layout(border='dashed 2px gray'))

hboxsys=wid.HBox([potV,gerar])
vboxsys=wid.VBox([xx,N,hboxsys,POTSOPT])

infovbox=wid.VBox([DIAG,leg_info,outtxt,outtsys])
plotbox=wid.VBox([PLOT,outt])
PLOT.children[6].button_style='info'

##########.####Accordion##############
ACCORDS=wid.Accordion(children=[vboxsys,infovbox])
ACCORDS.set_title(0,'Gerar Sistema')
ACCORDS.set_title(1,'Informações do Sistema Gerado/Diagonalizar')

################################TABS################################
tab=wid.Tab(children=[ACCORDS,plotbox])
tab.set_title(0, 'Gerar o Sistema')
tab.set_title(1, 'Funções de Onda')
PLOT.children[6].description=('Plotar')
display(tab)

***
# Unidades atômicas de Rydberg
<details><summary>Clique aqui para mostrar</summary>
Tabela de conversão de unidades entre o sistema de Rydberg e o MKSA, que é a base constante do SI. Cheque a fonte original mais abaixo para um tabela mais completa em inglês.

Fonte da tabela: [(Departamento de Física, Universidade Rei Fahd de Petróleos e Minerais)](https://faculty.kfupm.edu.sa/phys/aanaqvi/rydberg.pdf)

|Dimensão |  Sistema de Unidades Rydberg | MKSA (base do SI) | Outras equivalências|
| :- |  :-: | :-: | :-:|
|Comprimento | $$1=a_0$$ | $$5.2917725 \times 10^{-11} $$ | $$\approx 0.5291772 \text{ Å}$$|
|Massa| $$1=2m_e$$ | $$1.8218779\times 10^{-30} \text{ kg} $$| - |
|Tempo |$$1$$| $$1 4.8377687 \times 10^{-17} \text{ s}$$ | $$=1/20 \text{ fs}$$ |
|Frequência | $$1$$ | $$ 2.0670687 \times 10^{16} \text{ Hz}$$ | - |
|Velocidade| $$1$$ | $$1.0938457 \times 10^{6} \text{ m/s}$$| -|
|Momento| $$1$$ | $$1.9928534 \times 10^{-24} \text{ kg.m/s}$$| - |
|Energia| $$1=E_{Ryd}=e^2/(2a_0)$$ | $$2.1798741 \times 10 ^{-18} \text{ J}$$| $$3.605698 \text{ eV}$$|
|Carga | $$1=e/\sqrt{2}$$ | $$1.1329105\times 10^{-19} \text{ C}$$| - |
|Campo Elétrico |$$1$$| $$1 3.6360903 \times 10^{11} \text{ V/m}$$|-|
|Potencial | $$1$$ | $$19.241363 \text{ V}$$|-|

|Constantes Úteis | Símbolo |Sistema de Unidades Rydberg | MKSA (base do SI) |
|:-|:-:|:-:|:-|
|Constante de Planck reduzida | $$\hbar$$| $$1$$| $$1.05457266\times 10 ^{-34} \text{ J.s}$$|
|$h= 2\pi\hbar$| $$h$$ | $$2\pi$$ | $$6.6260755\times 10^{-34} \text{ J.s}$$|
|Carga do elétron | $$e$$ | $$\sqrt{2}$$| $$1.60217733\times 10 ^{-19} \text{ C}$$|
|Raio de Bohr|$$a_0$$| $$1=\hbar^2/(me^2)$$| $$5.29177249\times 10^{-11} \text{ m}$$|
|Energia de Hartree| $$E_h$$|$$2=e^2/(a_0)$$ | $$4.3597482\times 10^{-18} \text{ J}$$|
|Constante de estrutura fina| $$\alpha$$| $$e^2/(\hbar c)=1/137.0359895 $$| - |
|Velocidade da luz | $$c$$ | $$2/\alpha \approx 274$$ | $$2.99792458\times 10^{8} \text{ m/s}$$|
|Permeabilidade do vácuo| $$\mu_0$$ | $$\pi\alpha^2\approx 1.7 \times 10^{-4}$$| $$4\pi \times 10^{-7} \text{ H/m}$$|
|Permissividade do vácuo| $$\epsilon_0$$| $$1/(4\pi)\approx 0.08$$| $$8.854187817\times 10^{-12} \text{ F/m}$$|
|Massa do elétron| $$m_e$$| $$1/2$$ |$$9.1093897 \times 10^{-31} \text{ kg}$$|
|Massa do próton| $$m_p$$ | $$918.07638$$ | $$1.6726231\times 10^{-27} kg$$|
</details>

# Modelo Matemático
<details><summary>Clique aqui para mostrar</summary>
Escrevemos a equação de Schrodinger 1D independente do tempo, para um único eletron, fazendo $\hbar^2 /(2m)=1$, já que utilizaremos o sistema de unidades atômicas de Rydberg:
$$-\frac{\partial^2 \Psi}{\partial x^2}+V(x)=E\Psi \tag{1}$$
    
Discretizamos as derivadas parciais utilizando o método de diferenças finitas, fazendo a primeira derivada pela direita e a segunda pela esquerda para diminuir o erro:

$$\frac{\partial \Psi}{\partial x} \approx \frac{\Psi(x+\Delta x)-\Psi(x)}{\Delta x}$$

$$\frac{\partial \Psi}{\partial x} \approx \frac{\Psi(x)-\Psi(x-\Delta x)}{\Delta x}$$
    
Isso resultará em
    
$$\frac{\partial^2 \Psi}{\partial x^2} \approx \frac{\Psi(x+\Delta x)-2\Psi(x)+\Psi(x-\Delta x)}{\Delta x^2} \tag{2}$$

Descreveremos o sistema dentro de um intervalo que vai de $x_{min}$ até um certo valor $x_{max}$. Como o intervalo será discreto, utilizaremos um vetor posição $x$ com $N$ componentes para representá-lo. Por conveniência, mudamos a $\color{blue}{\text{notação com a variável }x}$ para $\color{red}{\text{notação com índices }i}$, tal que o vetor $x$ seja
    
$$x=\color{blue}{\begin{bmatrix}x_{min}&+&(1-1)\Delta x\\ x_{min}&+&(2-1)\Delta x \\ x_{min}&+& (3-1) \Delta x \\\vdots & +&\vdots\\ x_{min}&+&(N-1)\Delta x  \end{bmatrix}}=\color{red}{\begin{bmatrix}x_{1}\\ x_{2} \\ x_{3}\\\vdots  \\ x_{N}\end{bmatrix}}$$

Agora, de maneira similar, a função de onda será representada por um vetor $\Psi$ contendo todos os valores discretos de uma função de onda unidimensional representada por $N$ pontos e com uma indexação de que vai $i=1$ até $i=N$, ou seja:

$$\Psi=\color{blue}{\begin{bmatrix}\Psi(x + (1-1)\Delta x)\\\Psi(x + (2-1) \Delta x) \\\Psi( x+ (3-1) \Delta x) \\\vdots \\ \Psi(x + (N-1)\Delta x)\end{bmatrix}}= \color{red}{\begin{bmatrix}\Psi_{1}\\\Psi_{2}\\\Psi_3\\\vdots \\ \Psi_{N}\end{bmatrix}}$$
   
Substituindo (2) em (1) e utilizando a notação com índices, temos:

$$\frac{-\left(\Psi_{i-1}-2\Psi_{i}+\Psi_{i+1}\right)}{\Delta x^2}+V_i \Psi_i=E\Psi_i$$

Definindo $a={-1}/{\Delta x ^2}$, tem-se
    
$$a\left(\Psi_{i-1}-2\Psi_{i}+\Psi_{i+1}\right)+V_i \Psi_i=E\Psi_i$$

Rearranjando a equação acima de uma forma conveniente, obtem-se
    
$$a\Psi_{i-1}+ (V_1-2a)\Psi_{i}+ a\Psi_{i+1}=E\Psi_i$$

Isso pode ser reescrito como uma equação matricial da forma:
    
$$\begin{matrix}
a\Psi_{0}&+& (V_1-2a)\Psi_{1}&+& a\Psi_{2}  &= & E\Psi_1 \\ 
a \Psi_{1}&+&(V_2-2a)\Psi_{2}&+&a\Psi_{3}& = & E\Psi_2 \\ 
a \Psi_{2}&+&(V_3-2a)\Psi_{3}&+&a\Psi_{4}& = & E\Psi_3 \\ 
\vdots & +& \vdots&+& \vdots & = &\vdots\\ 
a\Psi_{N-1}&+&(V_{N}-2a)\Psi_{N}&+&a\Psi_{N+1}& = & E\Psi_{N}
\end{matrix}$$

Note que na primeira e última equações do sistema acima, do lado esquerdo da igualdade, temos os termos $\Psi_0$ e $\Psi_{N+1}$, que não correspondem a nenhuma componente do nosso vetor da função de onda discreta $\Psi$. Nós iremos utilizá-los para definir as condições de contorno (o valor da função de onda nas bordas do sistema).
    
>## Condições de contorno
> Abaixo estão dois exemplos básicos de condições de contorno
>### Fronteiras Fixas (poço infinito)
>Nessa condição forçamos a função de onda a ser 0 nas bordas, ou seja
>$$\Psi(x_{min})=\Psi(x_{max})=0$$
>Aqui, isso será equivalente a
>\begin{aligned}\Psi_0&=0 \\ \Psi_{N+1}&=0\end{aligned}
>### Condição de contorno periódica
>Nessa condição forçamos a função de onda no último ponto a ser igual ao valor do primeiro ponto, ou seja
>$$\Psi(x_{min})=\Psi_(x_{max})$$
>Aqui, isso será equivalente a
>\begin{aligned}\Psi_{N+1}&=\Psi_{1} \\ \Psi_{0}&=\Psi_{N}\end{aligned}
## Equação matricial
    
Aplicando a condição de contorno do poço infinito, temos:
$$\begin{matrix}
0&+& \color{blue}{(V_1-2a)}\Psi_{1}&+& \color{blue}a \Psi_{2}  &= & E\Psi_1 \\ 
\color{red}a \Psi_{1}&+&\color{red}{(V_2-2a)}\Psi_{2}&+&\color{red}a\Psi_{3}& = & E\Psi_2 \\ 
\color{orange}a \Psi_{2}&+&\color{orange}{(V_3-2a)}\Psi_{3}&+&\color{orange}a \Psi_{4}& = & E\Psi_3 \\ 
\vdots & +& \vdots&+& \vdots & = &\vdots\\ 
\color{purple}a\Psi_{N-2}&+&\color{purple}{(V_{N-1}-2a)} \Psi_{N-1}& +&\color{purple}a \Psi_{N}&= & E\Psi_{N-1}\\
\color{green}a\Psi_{N-1}&+&\color{green}{(V_{N}-2a)} \Psi_{N}&+&0& = & E\Psi_{N}
\end{matrix}$$

Esse sistema pode ser reescrito como uma matriz do tipo:
$$\begin{bmatrix}  \color{blue}{(V_1-2a)} & \color{blue}a & 0  & 0 & 0 & 0\\
\color{red}a & \color{red}{(V_2-2a)} & \color{red}a  & 0 & 0 &0 \\
0&\color{orange}a & \color{orange}{(V_3-2a)} &  \color{orange}a  & 0  &0  \\
 0 & 0 & \ddots & \ddots & \ddots & 0 \\
0& 0 & 0  & \color{purple}a & \color{purple}{(V_{N-1}-2a)}& \color{purple}a\\
0& 0 & 0 & 0 & \color{green}a & \color{green}{(V_{N}-2a)} 
\end{bmatrix} 
\begin{bmatrix}\Psi_{1}\\\Psi_2\\\Psi_3\\\vdots \\\Psi_{N-1}\\ \Psi_{N}\end{bmatrix}=E\begin{bmatrix}\Psi_{1}\\\Psi_2\\\Psi_3\\\vdots \\ \Psi_{N-1}\\\Psi_{N}\end{bmatrix}$$

Isso é equivalente a um problema de autovetores e autovalores, onde a matriz quadrada de ordem $N$ acima é o operador Hamiltoniano $\hat{H}$:

$$\hat{H}\Psi=E\Psi$$

Podemos diagonalizá-lo para obter os autovalores e seus respectivos autovetores. Os autovetores serão vetores $\Psi_n$ com $N$ componentes representando as respectivas funções de ondas dos estados $n$, e os autovalores $E$ serão as energias correspondentes aos respectivos estados. 
</details>
